In [3]:
import cv2 
import os
import numpy as np
from scipy import ndimage
from sklearn.model_selection import train_test_split


# 获取两位小数 四舍五不入
def Get_Two_Float(num, n):
    num = str(num)     #转为字符串
    a, b, c = num.partition('.') # 以.号为分隔符 b = .
    c = (c+"0"*n)[:n]       # 不论传入的函数有几位小数，在字符串后面都添加n位小数0，取两位小数
    x = ".".join([a, c])    
    return float(x)


# 缩放及扩充图像
def Small(newimage,size):
    
    # 将缩放图像尺寸接近256*256
    y = size / newimage.shape[0]   # 纵向需要缩放y倍到达256
    x = size / newimage.shape[1]   # 横向需要缩放x倍到达256
    y = Get_Two_Float(y,2)        # 取两位小数
    x = Get_Two_Float(x,2)
    new = ndimage.zoom(newimage,(y,x,1), order=0)  #缩放图像
    
    # 计算缩放后图像各轴长度与256的差值
    y_pad = size - new.shape[0]
    x_pad = size - new.shape[1]
        
    # 计算y、x轴两边需填充的长度
    lower_ypad = y_pad//2
    upper_ypad = y_pad - lower_ypad
    lower_xpad = x_pad//2
    upper_xpad = x_pad - lower_xpad
    
    # 填充图像尺寸为256*256
    new1 = np.pad(new,((lower_ypad,upper_ypad),
                       (lower_xpad,upper_xpad),
                       (0,0)),mode='constant',constant_values=255)
    return new1

# 获取图像地址
def Get_PicPath(path):
    paths = []   # 用来存放图像地址
    path_dir = os.listdir(path) #获取path路径下的文件
    for p in path_dir:
        img_path=path+'/'+p     #获取图片路径
        paths.append(img_path)
    return paths

# 剪裁图片
def Cut_Pic(path,save_path):
    paths = Get_PicPath(path)
    z = 0
    for l in range(z,len(paths)):  
        #print(l)
        img = cv2.imread(paths[l])  # 循环读取图片
        imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转灰度图
        
        ret,thresh = cv2.threshold(imgray,127,255,0)   # 得到二值图像
        
        #OpenCV定义的结构元素
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
        #腐蚀膨胀
        erode=cv2.erode(thresh,kernel,iterations=1)
        dilate=cv2.dilate(erode,kernel,iterations=1)
        
        h = dilate.shape[0] 
        w = dilate.shape[1] 
        x = []
        y = []
        for k in range(h):     # 遍历二值图像获取像素值
            for j in range(w):
                if thresh[k][j] == 0:
                    if j > 240 and j < 1450:
                        if k > 10 and k < 1000:  # 根据图片设定剪裁区域区域
                            x.append(j)
                            y.append(k)
        x1 = min(x)     # 得到最小像素值和最大像素值坐标，防止剪切到图片内容，去掉多余部分
        x2 = max(x)
        y1 = min(y)
        y2 = max(y)

        newimage = img[y1-10:y2,x1:x2+100]   # 剪裁图片

        z += 1
        print(paths[l])
        new = Small(newimage,128)    # 缩放图片
        cv2.imwrite(save_path + '/%03d'%l + '.jpg',new)

# 将图片背景换为白底
def Change_Bk(path,save_path):
    paths = Get_PicPath(path)
    z = 0
    for path in paths:
        print(path)
        img = cv2.imread(path)
        
        #lower_white = np.array([69, 79, 89], dtype=np.uint8)
        lower_white = np.array([110, 120, 130], dtype=np.uint8)        
        upper_white = np.array([255, 255, 255], dtype=np.uint8)
        mask = cv2.inRange(img, lower_white, upper_white)   # 将阈值内背景换为白底
        #res = cv2.bitwise_not(img, img, mask)
        #OpenCV定义的结构元素
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
        #腐蚀膨胀
        erode=cv2.erode(mask,kernel,iterations=1)
        dilate=cv2.dilate(erode,kernel,iterations=1)
        
        coloured = img.copy()
        coloured[dilate == 255] = (255, 255, 255)
        cv2.imwrite(save_path + '/%03d'%z + '.jpg',coloured)
        z += 1

In [18]:
path = r"C:/Users/Administrator/Desktop/cut_mango/0"
save_path = r"C:/Users/Administrator/Desktop/cut_mango/0"
paths = []
path_dir = os.listdir(path)
z = 0
for p in path_dir:
    #img1=os.listdir(path+'/'+p) ##img为对应路径下的文件夹
    #for i in img1:
    img_path=path+'/'+p ##是的话获取图片路径
    paths.append(img_path)
    for i in range(z,len(paths),1):
        img = cv2.imread(paths[i])
        new = Small(img,128)
        cv2.imwrite(save_path + '/%03d'%i + '.jpg',new)
        z += 1

In [4]:
path = r"C:/Users/Administrator/Desktop/mango/8"
save_path = r"C:/Users/Administrator/Desktop/cut_mango/1"
Cut_Pic(path,save_path)

C:/Users/Administrator/Desktop/mango/8/001.jpg
C:/Users/Administrator/Desktop/mango/8/002.jpg
C:/Users/Administrator/Desktop/mango/8/003.jpg
C:/Users/Administrator/Desktop/mango/8/004.jpg
C:/Users/Administrator/Desktop/mango/8/005.jpg
C:/Users/Administrator/Desktop/mango/8/006.jpg
C:/Users/Administrator/Desktop/mango/8/007.jpg
C:/Users/Administrator/Desktop/mango/8/008.jpg
C:/Users/Administrator/Desktop/mango/8/009.jpg
C:/Users/Administrator/Desktop/mango/8/010.jpg
C:/Users/Administrator/Desktop/mango/8/011.jpg
C:/Users/Administrator/Desktop/mango/8/012.jpg
C:/Users/Administrator/Desktop/mango/8/013.jpg
C:/Users/Administrator/Desktop/mango/8/014.jpg
C:/Users/Administrator/Desktop/mango/8/015.jpg
C:/Users/Administrator/Desktop/mango/8/016.jpg
C:/Users/Administrator/Desktop/mango/8/017.jpg
C:/Users/Administrator/Desktop/mango/8/018.jpg
C:/Users/Administrator/Desktop/mango/8/019.jpg
C:/Users/Administrator/Desktop/mango/8/020.jpg
C:/Users/Administrator/Desktop/mango/8/021.jpg
C:/Users/Admi

C:/Users/Administrator/Desktop/mango/8/176.jpg
C:/Users/Administrator/Desktop/mango/8/177.jpg
C:/Users/Administrator/Desktop/mango/8/178.jpg
C:/Users/Administrator/Desktop/mango/8/179.jpg
C:/Users/Administrator/Desktop/mango/8/180.jpg
C:/Users/Administrator/Desktop/mango/8/181.jpg
C:/Users/Administrator/Desktop/mango/8/182.jpg
C:/Users/Administrator/Desktop/mango/8/183.jpg
C:/Users/Administrator/Desktop/mango/8/184.jpg
C:/Users/Administrator/Desktop/mango/8/185.jpg
C:/Users/Administrator/Desktop/mango/8/186.jpg
C:/Users/Administrator/Desktop/mango/8/187.jpg
C:/Users/Administrator/Desktop/mango/8/188.jpg
C:/Users/Administrator/Desktop/mango/8/189.jpg
C:/Users/Administrator/Desktop/mango/8/190.jpg
C:/Users/Administrator/Desktop/mango/8/191.jpg
C:/Users/Administrator/Desktop/mango/8/192.jpg
C:/Users/Administrator/Desktop/mango/8/193.jpg
C:/Users/Administrator/Desktop/mango/8/194.jpg
C:/Users/Administrator/Desktop/mango/8/195.jpg
C:/Users/Administrator/Desktop/mango/8/196.jpg
C:/Users/Admi

C:/Users/Administrator/Desktop/mango/8/351.jpg
C:/Users/Administrator/Desktop/mango/8/352.jpg
C:/Users/Administrator/Desktop/mango/8/353.jpg
C:/Users/Administrator/Desktop/mango/8/354.jpg
C:/Users/Administrator/Desktop/mango/8/355.jpg
C:/Users/Administrator/Desktop/mango/8/356.jpg
C:/Users/Administrator/Desktop/mango/8/357.jpg
C:/Users/Administrator/Desktop/mango/8/358.jpg
C:/Users/Administrator/Desktop/mango/8/359.jpg
C:/Users/Administrator/Desktop/mango/8/360.jpg
C:/Users/Administrator/Desktop/mango/8/361.jpg
C:/Users/Administrator/Desktop/mango/8/362.jpg
C:/Users/Administrator/Desktop/mango/8/363.jpg
C:/Users/Administrator/Desktop/mango/8/364.jpg
C:/Users/Administrator/Desktop/mango/8/365.jpg
C:/Users/Administrator/Desktop/mango/8/366.jpg
C:/Users/Administrator/Desktop/mango/8/367.jpg
C:/Users/Administrator/Desktop/mango/8/368.jpg
C:/Users/Administrator/Desktop/mango/8/369.jpg
C:/Users/Administrator/Desktop/mango/8/370.jpg
C:/Users/Administrator/Desktop/mango/8/371.jpg
C:/Users/Admi

C:/Users/Administrator/Desktop/mango/8/526.jpg
C:/Users/Administrator/Desktop/mango/8/527.jpg
C:/Users/Administrator/Desktop/mango/8/528.jpg
C:/Users/Administrator/Desktop/mango/8/529.jpg
C:/Users/Administrator/Desktop/mango/8/530.jpg
C:/Users/Administrator/Desktop/mango/8/531.jpg
C:/Users/Administrator/Desktop/mango/8/532.jpg
C:/Users/Administrator/Desktop/mango/8/533.jpg
C:/Users/Administrator/Desktop/mango/8/534.jpg
C:/Users/Administrator/Desktop/mango/8/535.jpg
C:/Users/Administrator/Desktop/mango/8/536.jpg
C:/Users/Administrator/Desktop/mango/8/537.jpg
C:/Users/Administrator/Desktop/mango/8/538.jpg
C:/Users/Administrator/Desktop/mango/8/539.jpg
C:/Users/Administrator/Desktop/mango/8/540.jpg
C:/Users/Administrator/Desktop/mango/8/541.jpg
C:/Users/Administrator/Desktop/mango/8/542.jpg
C:/Users/Administrator/Desktop/mango/8/543.jpg
C:/Users/Administrator/Desktop/mango/8/544.jpg
C:/Users/Administrator/Desktop/mango/8/545.jpg
C:/Users/Administrator/Desktop/mango/8/546.jpg
C:/Users/Admi

C:/Users/Administrator/Desktop/mango/8/701.jpg
C:/Users/Administrator/Desktop/mango/8/702.jpg
C:/Users/Administrator/Desktop/mango/8/703.jpg
C:/Users/Administrator/Desktop/mango/8/704.jpg
C:/Users/Administrator/Desktop/mango/8/705.jpg
C:/Users/Administrator/Desktop/mango/8/706.jpg
C:/Users/Administrator/Desktop/mango/8/707.jpg
C:/Users/Administrator/Desktop/mango/8/708.jpg
C:/Users/Administrator/Desktop/mango/8/709.jpg
C:/Users/Administrator/Desktop/mango/8/710.jpg
C:/Users/Administrator/Desktop/mango/8/711.jpg
C:/Users/Administrator/Desktop/mango/8/712.jpg
C:/Users/Administrator/Desktop/mango/8/713.jpg
C:/Users/Administrator/Desktop/mango/8/714.jpg
C:/Users/Administrator/Desktop/mango/8/715.jpg
C:/Users/Administrator/Desktop/mango/8/716.jpg
C:/Users/Administrator/Desktop/mango/8/717.jpg
C:/Users/Administrator/Desktop/mango/8/718.jpg
C:/Users/Administrator/Desktop/mango/8/719.jpg
C:/Users/Administrator/Desktop/mango/8/720.jpg
C:/Users/Administrator/Desktop/mango/8/721.jpg
C:/Users/Admi